In [108]:
# Generate Multiline Vector from Labelbox Labels

from glob import glob
import pandas as pd
import json
import rasterio as rio
from shapely.geometry import LineString
from geopandas import GeoDataFrame

# Extract Chips

import gdal
import os


# Exporting Intersecting Chips

from osgeo import gdal, osr, ogr
from shutil import copyfile

Help on function Translate in module osgeo.gdal:

Translate(destName, srcDS, **kwargs)
    Convert a dataset.
    Arguments are :
      destName --- Output dataset name
      srcDS --- a Dataset object or a filename
    Keyword arguments are :
      options --- return of gdal.TranslateOptions(), string or array of strings
      other keywords arguments of gdal.TranslateOptions()
    If options is provided as a gdal.TranslateOptions() object, other keywords are ignored.



In [ ]:
# Extract Chips from Polygons

def extract_chips(input_dir,output_dir,tile_size_x,tile_size_y):
 
    tile_size_x = 100
    tile_size_y = 100


    for in_path in input_files: 
        ds = gdal.Open(in_path)
        band = ds.GetRasterBand(1)
        xsize = band.XSize
        ysize = band.YSize

        input_filename = in_path.split("/")[-1]
        image_id = in_path.split("/")[-1].split(".")[0]
        output_filename = image_id + "_"
        out_path = out_path_base + image_id + "/"

        # Check if directory exists
        if not os.path.isdir(out_path):
            os.mkdir(out_path)


        for i in range(0, xsize, tile_size_x):
            for j in range(0, ysize, tile_size_y):
                gdal.Translate(str(out_path + output_filename + str(i) + "_" + str(j) + ".tif"),in_path,
                               format="GTiff",srcWin=(str(i),str(j),str(tile_size_x),str(tile_size_y)))


chipping 1.tif
/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Individual_Polygons/TIF/1.tif /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Chips/test/1/1_00
/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Individual_Polygons/TIF/1.tif /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Chips/test/1/1_0100
/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Individual_Polygons/TIF/1.tif /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Chips/test/1/1_0200
/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Individual_Polygons/TIF/1.tif /Volumes/Lacie/zhenyadata/Project_Can

In [ ]:
input_files = glob("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Individual_Polygons/TIF/*") 
out_path_base = '/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Chips/test/'
tile_size_x = 100
tile_size_y = 100

extract_chips(input_dir,output_dir,tile_size_x,tile_size_y)


In [90]:
# Generate Multiline Vector and Export to Shapefile

csv_loc = "/Users/purgatorid/Documents/GitHub/canopy-gis/data_collection/data/labelled/labelbox_test.csv"
tif_file_list = glob("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Individual_Polygons/TIF/*")
output_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/test_line_labelling/multilines/"

df = pd.read_csv(csv_loc)


polyline_objs = {}
for parent_index in range(len(df)):
    polyline_list = []
    json_obj = json.loads(df["Label"][parent_index])["objects"]
    image_id = df["External ID"][parent_index].split(".")[0]
    matching_tif = [i for i in tif_file_list if i.split("/")[-1].split(".")[0] == image_id][0]
    affine = rio.open(matching_tif).transform
    for index_2 in range(len(json_obj)):
        temp_list = [] 
        try:
            coords_list = json_obj[index_2]["line"]
            for coords in coords_list:
                latlon = affine * (coords["x"],coords["y"])
                temp_list.append(latlon)
        except:
            continue
            
        polyline_objs.setdefault(image_id,[]).append(LineString(temp_list))
        
for geo_id in polyline_objs.keys():
    multiple_line_df = GeoDataFrame(polyline_objs[geo_id])
    multiple_line_df.rename(columns = {0:"geometry"},inplace=True)
    multiple_line_df.set_geometry(col='geometry', inplace=True)
    multiple_line_df.to_file(output_dir + geo_id + ".shp", driver='ESRI Shapefile')

In [111]:
# Export Chips Intersecting with Multiline Vectors


dir_list = glob("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Chips/full/*")
vector_list = glob("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/test_line_labelling/multilines/*.shp")
output_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Chips/yes/"

for vector_path in vector_list:
    vec_id = vector_path.split("/")[-1].split(".")[0]
    vector = ogr.Open(vector_path)
    # Get vector geometry
    layer = vector.GetLayer()
    feature_count = vector.GetLayer().GetFeatureCount()
    for feature_number in range(feature_count):
        feature = layer.GetFeature(feature_number)
        vectorGeometry = feature.GetGeometryRef()
        for in_dir in dir_list:
            tif_id = in_dir.split("/")[-1]
            if tif_id == vec_id:
                raster_list = glob(in_dir + "/*")
                for raster_path in raster_list:
                    raster = gdal.Open(raster_path)
                    # Get raster geometry
                    transform = raster.GetGeoTransform()
                    pixelWidth = transform[1]
                    pixelHeight = transform[5]
                    cols = raster.RasterXSize
                    rows = raster.RasterYSize

                    xLeft = transform[0]
                    yTop = transform[3]
                    xRight = xLeft+cols*pixelWidth
                    yBottom = yTop+rows*pixelHeight

                    ring = ogr.Geometry(ogr.wkbLinearRing)
                    ring.AddPoint(xLeft, yTop)
                    ring.AddPoint(xLeft, yBottom)
                    ring.AddPoint(xRight, yBottom)
                    ring.AddPoint(xRight, yTop)
                    ring.AddPoint(xLeft, yTop)
                    rasterGeometry = ogr.Geometry(ogr.wkbPolygon)
                    rasterGeometry.AddGeometry(ring)

                    # If intersection detected, copy chip to destination directory
                    if rasterGeometry.Intersect(vectorGeometry):
        
                        # Check if directory exists
                        if not os.path.isdir(output_path + tif_id):
                            os.mkdir(output_path + tif_id)

                        out_file = output_path + tif_id + "/" + raster_path.split("/")[-1] + ".tif"
                        copyfile(raster_path,out_file)
                        raster_list.remove(raster_path)